In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


##Read Data

In [ ]:
import json
import pandas as pd
from google.colab import userdata

train_path = "/content/drive/MyDrive/NLP Project/data/train_cleaned.jsonl"
test_path = "/content/drive/MyDrive/NLP Project/data/test_cleaned.jsonl"


df_train = pd.read_json(train_path,lines=True)
df_test = pd.read_json(test_path, lines=True)

df_train.head()
df_test.head()


,labNo,taskNo,questioner,question,code,startLine,endLine,questionType,answer
0,9,1,TA,"In Task 2, explain the composition of the crea...",import random\nfrom cs1graphics import *\n\nim...,17.0,28.0,variable,"In task2, self.suit stores the card's pattern ..."
1,6,1,student,when doing count_integer\nCan't we compare fro...,"import random\n\n\ndef drawing_integers(lb, ub...",NaN,NaN,task,Only counts all natural numbers from the minim...
2,8,0,student,"I'm sorry, but was the part you pointed out fo...",from time import sleep\n\ndef merge(input_file...,NaN,NaN,task,I don't have to take the lecture. Didn't you s...
3,8,0,student,```\nfor i in range input_filenames:\n ...,from time import sleep\n\ndef merge(input_file...,6.0,7.0,usage,Yes~
4,7,0,TA,Please explain the function dec_to_any_string ...,import math\n\n#------------------------------...,28.0,35.0,variable,"In the case of dec_to_any_string, an empty str..."


In [ ]:
# Filter out rows with questionType 'usage' or 'code_explain'
df_train = df_train[~df_train['questionType'].isin(['code_understanding'])].reset_index(drop=True)
df_test = df_test[~df_test['questionType'].isin(['code_understanding'])].reset_index(drop=True)

In [ ]:
#Create labels
classes = df_train["questionType"].unique()

In [ ]:
df_train.columns

Index(['labNo', 'taskNo', 'questioner', 'question', 'code', 'startLine',
       'endLine', 'questionType', 'answer'],
      dtype='object')

In [ ]:
df_test.columns

Index(['labNo', 'taskNo', 'questioner', 'question', 'code', 'startLine',
       'endLine', 'questionType', 'answer'],
      dtype='object')

##TFIDF Vectorizer + Random Forest

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Assuming df_train and df_test are already defined
# Input = 'question', Output = 'questionType'

# Train/validation split on df_train
X_train = df_train['question']
y_train = df_train['questionType']

# Test set
X_test = df_test['question']
y_test = df_test['questionType']

# Define pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('rf', RandomForestClassifier(n_estimators=500, random_state=42))
])

# Train model
pipeline.fit(X_train, y_train)

# Predict and evaluate on df_test
y_pred = pipeline.predict(X_test)
print("=== Evaluation on df_test ===")
print(classification_report(y_test, y_pred))


=== Evaluation on df_test ===
              precision    recall  f1-score   support

   algorithm       0.50      0.54      0.52       239
code_explain       0.86      0.65      0.74       188
       error       0.84      0.64      0.73       193
     logical       0.61      0.60      0.60       211
   reasoning       0.83      0.79      0.81       160
        task       0.46      0.67      0.55       264
       usage       0.33      0.09      0.14       162
    variable       0.77      0.93      0.85       389

    accuracy                           0.65      1806
   macro avg       0.65      0.61      0.62      1806
weighted avg       0.65      0.65      0.64      1806



##Count Vectorizer + Random Forest

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Input-output
X_train = df_train['question']
y_train = df_train['questionType']
X_test = df_test['question']
y_test = df_test['questionType']

# Define pipeline with CountVectorizer
pipeline = Pipeline([
    ('count_vec', CountVectorizer()),  # Basic bag-of-words
    ('rf', RandomForestClassifier(n_estimators=500, random_state=42))
])

# Train model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
print("=== Evaluation with CountVectorizer ===")
print(classification_report(y_test, y_pred))


=== Evaluation with CountVectorizer ===
              precision    recall  f1-score   support

   algorithm       0.53      0.59      0.56       239
code_explain       0.88      0.74      0.80       188
       error       0.83      0.63      0.71       193
     logical       0.63      0.60      0.61       211
   reasoning       0.80      0.80      0.80       160
        task       0.48      0.70      0.57       264
       usage       0.56      0.17      0.26       162
    variable       0.81      0.91      0.85       389

    accuracy                           0.68      1806
   macro avg       0.69      0.64      0.65      1806
weighted avg       0.69      0.68      0.67      1806



##BERT Base Uncased (No finetuning) + Random Forest

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()  # Inference mode

# Function to extract CLS token embeddings
def get_bert_embeddings(text_list):
    embeddings = []
    with torch.no_grad():
        for text in tqdm(text_list, desc="Embedding texts"):
            inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
            outputs = bert_model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)  # CLS token
            embeddings.append(cls_embedding.numpy())
    return embeddings

# Prepare data
X_train_texts = df_train['question'].tolist()
y_train = df_train['questionType']
X_test_texts = df_test['question'].tolist()
y_test = df_test['questionType']

# Get embeddings
X_train_embeddings = get_bert_embeddings(X_train_texts)
X_test_embeddings = get_bert_embeddings(X_test_texts)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_embeddings, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_embeddings)
print("=== Evaluation with BERT embeddings + Random Forest ===")
print(classification_report(y_test, y_pred))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Embedding texts: 100%|██████████| 1806/1806 [01:11<00:00, 25.29it/s]


=== Evaluation with BERT embeddings + Random Forest ===
              precision    recall  f1-score   support

   algorithm       0.30      0.31      0.31       239
code_explain       0.69      0.54      0.61       188
       error       0.48      0.56      0.52       193
     logical       0.38      0.33      0.35       211
   reasoning       0.76      0.42      0.54       160
        task       0.41      0.62      0.50       264
       usage       0.51      0.22      0.30       162
    variable       0.72      0.83      0.77       389

    accuracy                           0.52      1806
   macro avg       0.53      0.48      0.49      1806
weighted avg       0.54      0.52      0.51      1806



##DistilBert (No finetuning) + Random Forest


In [ ]:
#Distillbert
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm

# Load pre-trained BERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
db_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
db_model.eval()  # Inference mode

# Function to extract CLS token embeddings
def get_bert_embeddings(text_list):
    embeddings = []
    with torch.no_grad():
        for text in tqdm(text_list, desc="Embedding texts"):
            inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
            outputs = db_model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)  # CLS token
            embeddings.append(cls_embedding.numpy())
    return embeddings

# Prepare data
X_train_texts = df_train['question'].tolist()
y_train = df_train['questionType']
X_test_texts = df_test['question'].tolist()
y_test = df_test['questionType']

# Get embeddings
X_train_embeddings = get_bert_embeddings(X_train_texts)
X_test_embeddings = get_bert_embeddings(X_test_texts)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_embeddings, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_embeddings)
print("=== Evaluation with BERT embeddings + Random Forest ===")
print(classification_report(y_test, y_pred))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Embedding texts: 100%|██████████| 1806/1806 [00:36<00:00, 48.98it/s]


=== Evaluation with BERT embeddings + Random Forest ===
              precision    recall  f1-score   support

   algorithm       0.36      0.34      0.35       239
code_explain       0.65      0.60      0.62       188
       error       0.47      0.48      0.47       193
     logical       0.45      0.46      0.46       211
   reasoning       0.79      0.39      0.52       160
        task       0.40      0.58      0.47       264
       usage       0.56      0.19      0.29       162
    variable       0.71      0.86      0.78       389

    accuracy                           0.54      1806
   macro avg       0.55      0.49      0.50      1806
weighted avg       0.55      0.54      0.52      1806



##DistilBert (MLM finetuning using SOQA 25k datapoints) + Random Forest


In [ ]:
#Finetuned distillbert
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers import AutoModelForMaskedLM

# Load pre-trained BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased')
# bert_model.eval()  # Inference mode


# model_checkpoint = "distilbert-base-uncased"
# model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

model_name = "KJCHUA/distilbert-base-uncased-finetuned-soqa"

tokenizer = AutoTokenizer.from_pretrained(model_name)
distilbert_model = AutoModelForMaskedLM.from_pretrained(model_name)
distilbert_model.eval()

encoder = distilbert_model.distilbert
encoder.eval()
print(encoder)

# print(distilbert_model)
# Function to extract CLS token embeddings
def get_bert_embeddings(text_list):
    embeddings = []
    with torch.no_grad():
        for text in tqdm(text_list, desc="Embedding texts"):
            inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
            outputs = encoder(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)  # CLS token
            embeddings.append(cls_embedding.numpy())
    return embeddings

# Prepare data
X_train_texts = df_train['question'].tolist()
y_train = df_train['questionType']
X_test_texts = df_test['question'].tolist()
y_test = df_test['questionType']

# Get embeddings
X_train_embeddings = get_bert_embeddings(X_train_texts)
X_test_embeddings = get_bert_embeddings(X_test_texts)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_embeddings, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_embeddings)
print("=== Evaluation with BERT embeddings + Random Forest ===")
print(classification_report(y_test, y_pred))


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

Embedding texts: 100%|██████████| 1806/1806 [00:37<00:00, 48.51it/s]


=== Evaluation with BERT embeddings + Random Forest ===
              precision    recall  f1-score   support

   algorithm       0.34      0.39      0.37       239
code_explain       0.70      0.66      0.68       188
       error       0.59      0.54      0.56       193
     logical       0.55      0.56      0.55       211
   reasoning       0.75      0.41      0.53       160
        task       0.44      0.63      0.52       264
       usage       0.57      0.16      0.25       162
    variable       0.72      0.84      0.78       389

    accuracy                           0.57      1806
   macro avg       0.58      0.52      0.53      1806
weighted avg       0.58      0.57      0.56      1806



In [ ]:
#Finetuned distillbert
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers import AutoModelForMaskedLM

# Load pre-trained BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased')
# bert_model.eval()  # Inference mode


# model_checkpoint = "distilbert-base-uncased"
# model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

model_name = "KJCHUA/distilbert-base-uncased-finetuned-soqa"

tokenizer = AutoTokenizer.from_pretrained(model_name)
distilbert_model = AutoModelForMaskedLM.from_pretrained(model_name)
distilbert_model.eval()

encoder = distilbert_model.distilbert
encoder.eval()
print(encoder)

# print(distilbert_model)
# Function to extract CLS token embeddings
def get_bert_embeddings(text_list):
    embeddings = []
    with torch.no_grad():
        for text in tqdm(text_list, desc="Embedding texts"):
            inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
            outputs = encoder(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)  # CLS token
            embeddings.append(cls_embedding.numpy())
    return embeddings

# Prepare data
X_train_texts = df_train['question'].tolist()
y_train = df_train['questionType']
X_test_texts = df_test['question'].tolist()
y_test = df_test['questionType']

# Get embeddings
X_train_embeddings = get_bert_embeddings(X_train_texts)
X_test_embeddings = get_bert_embeddings(X_test_texts)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_embeddings, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_embeddings)
print("=== Evaluation with BERT embeddings + Random Forest ===")
print(classification_report(y_test, y_pred))


##DistilBert (MLM finetuning using SOQA all datapoints) + Random Forest

In [ ]:
#Finetuned distillbert 330k data points.
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers import AutoModelForMaskedLM

# Load pre-trained BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased')
# bert_model.eval()  # Inference mode


# model_checkpoint = "distilbert-base-uncased"
# model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

model_name = "KJCHUA/distilbert-base-uncased-finetuned-soqa-330k"

tokenizer = AutoTokenizer.from_pretrained(model_name)
distilbert_model = AutoModelForMaskedLM.from_pretrained(model_name)
distilbert_model.eval()

encoder = distilbert_model.distilbert
encoder.eval()
print(encoder)

# print(distilbert_model)
# Function to extract CLS token embeddings
def get_bert_embeddings(text_list):
    embeddings = []
    with torch.no_grad():
        for text in tqdm(text_list, desc="Embedding texts"):
            inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
            outputs = encoder(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)  # CLS token
            embeddings.append(cls_embedding.numpy())
    return embeddings

# Prepare data
X_train_texts = df_train['question'].tolist()
y_train = df_train['questionType']
X_test_texts = df_test['question'].tolist()
y_test = df_test['questionType']

# Get embeddings
X_train_embeddings = get_bert_embeddings(X_train_texts)
X_test_embeddings = get_bert_embeddings(X_test_texts)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_embeddings, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_embeddings)
print("=== Evaluation with BERT embeddings + Random Forest ===")
print(classification_report(y_test, y_pred))


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

Embedding texts: 100%|██████████| 1847/1847 [01:54<00:00, 16.07it/s]


=== Evaluation with BERT embeddings + Random Forest ===
                    precision    recall  f1-score   support

         algorithm       0.38      0.45      0.41       239
      code_explain       0.70      0.65      0.67       188
code_understanding       0.00      0.00      0.00        41
             error       0.55      0.46      0.50       193
           logical       0.47      0.50      0.49       211
         reasoning       0.64      0.50      0.56       160
              task       0.39      0.55      0.45       264
             usage       0.39      0.09      0.14       162
          variable       0.73      0.87      0.79       389

          accuracy                           0.54      1847
         macro avg       0.47      0.45      0.45      1847
      weighted avg       0.53      0.54      0.52      1847



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##ModernBERT no finetuning + Random Forest

In [ ]:
#Modernbert w/o our finetuning
import pandas as pd
import torch
from transformers import AutoTokenizer, ModernBertForMaskedLM
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers import AutoModelForMaskedLM

# Load pre-trained BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased')
# bert_model.eval()  # Inference mode


# model_checkpoint = "distilbert-base-uncased"
# model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

model_name = "answerdotai/ModernBERT-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
distilbert_model = ModernBertForMaskedLM.from_pretrained(model_name)
distilbert_model.eval()

encoder = distilbert_model.model
encoder.eval()
print(encoder)

# print(distilbert_model)
# Function to extract CLS token embeddings
def get_bert_embeddings(text_list):
    embeddings = []
    with torch.no_grad():
        for text in tqdm(text_list, desc="Embedding texts"):
            inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
            outputs = encoder(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)  # CLS token
            embeddings.append(cls_embedding.numpy())
    return embeddings

# Prepare data
X_train_texts = df_train['question'].tolist()
y_train = df_train['questionType']
X_test_texts = df_test['question'].tolist()
y_test = df_test['questionType']

# Get embeddings
X_train_embeddings = get_bert_embeddings(X_train_texts)
X_test_embeddings = get_bert_embeddings(X_test_texts)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_embeddings, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_embeddings)
print("=== Evaluation with BERT embeddings + Random Forest ===")
print(classification_report(y_test, y_pred))


ModernBertModel(
  (embeddings): ModernBertEmbeddings(
    (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (drop): Dropout(p=0.0, inplace=False)
  )
  (layers): ModuleList(
    (0): ModernBertEncoderLayer(
      (attn_norm): Identity()
      (attn): ModernBertAttention(
        (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
        (rotary_emb): ModernBertRotaryEmbedding()
        (Wo): Linear(in_features=768, out_features=768, bias=False)
        (out_drop): Identity()
      )
      (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): ModernBertMLP(
        (Wi): Linear(in_features=768, out_features=2304, bias=False)
        (act): GELUActivation()
        (drop): Dropout(p=0.0, inplace=False)
        (Wo): Linear(in_features=1152, out_features=768, bias=False)
      )
    )
    (1-21): 21 x ModernBertEncoderLayer(
      (attn_norm): LayerNorm((768,), eps=1e-05, e

Embedding texts: 100%|██████████| 1847/1847 [02:26<00:00, 12.58it/s]


=== Evaluation with BERT embeddings + Random Forest ===
                    precision    recall  f1-score   support

         algorithm       0.32      0.44      0.37       239
      code_explain       0.47      0.29      0.36       188
code_understanding       0.00      0.00      0.00        41
             error       0.38      0.34      0.36       193
           logical       0.32      0.28      0.30       211
         reasoning       0.36      0.17      0.24       160
              task       0.34      0.49      0.40       264
             usage       0.38      0.14      0.21       162
          variable       0.58      0.79      0.67       389

          accuracy                           0.42      1847
         macro avg       0.35      0.33      0.32      1847
      weighted avg       0.40      0.42      0.39      1847



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##DistilBert (Finetuning with MLM with all SOQA dataset and CS1QA data)

In [ ]:
!pip install datasets

In [ ]:
#Finetuning distillbert using labelled dataset

from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForMaskedLM
model_name = "KJCHUA/distilbert-base-uncased-finetuned-soqa"

tokenizer = AutoTokenizer.from_pretrained(model_name)
distilbert_model = AutoModelForMaskedLM.from_pretrained(model_name)
distilbert_model.eval()

encoder = distilbert_model.distilbert
encoder.eval()
print(encoder)

# Encode labels
label_encoder = LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train['questionType'])
df_test['label'] = label_encoder.transform(df_test['questionType'])

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(df_train[['question', 'label']])
test_dataset = Dataset.from_pandas(df_test[['question', 'label']])

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [ ]:
train_dataset

Dataset({
    features: ['question', 'label'],
    num_rows: 5543
})

In [ ]:
def tokenize_function(example):
    return tokenizer(example["question"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/5417 [00:00<?, ? examples/s]

Map:   0%|          | 0/1806 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_encoder.classes_)
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at KJCHUA/distilbert-base-uncased-finetuned-soqa and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ebitdaddy93 (ebitdaddy93-nus) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.791400,0.813035
2,0.423900,0.744312
3,0.301600,0.741491


TrainOutput(global_step=1017, training_loss=0.6826434733241709, metrics={'train_runtime': 211.8622, 'train_samples_per_second': 76.706, 'train_steps_per_second': 4.8, 'total_flos': 538239510153216.0, 'train_loss': 0.6826434733241709, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import classification_report

# Get predictions
predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(-1)
y_true = df_test['label'].values

# Classification report
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

   algorithm       0.60      0.63      0.61       239
code_explain       0.86      0.86      0.86       188
       error       0.80      0.72      0.76       193
     logical       0.61      0.68      0.64       211
   reasoning       0.89      0.86      0.88       160
        task       0.68      0.70      0.69       264
       usage       0.55      0.46      0.50       162
    variable       0.91      0.92      0.91       389

    accuracy                           0.75      1806
   macro avg       0.74      0.73      0.73      1806
weighted avg       0.75      0.75      0.75      1806

